In [1]:
from bs4 import BeautifulSoup
import requests
# import requests_async as requests
from fake_useragent import UserAgent
# import asyncio
def getHtml(method, url, params= None, data= None, encoding= 'utf-8'):
    try:
    # headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}
    # req = requests.request(req_type, url, data)
        headers = {'user-agent':str(UserAgent().random)}

        if method == 'get':
            req = requests.get(url, headers= headers, params= params)
        elif method == 'post':
            req = requests.post(url, headers= headers, data= data)
            
        # req=requests.get(url, headers= headers, params= params)
        if req.status_code == 200:
            print('requests -> 200')
            req.encoding = encoding
            return BeautifulSoup(req.text, 'html.parser')
        print('requests -> ', req.status_code)
        return None
    except:
        return None
    
def textClean(splitChar='', text='', loop=1, reRight=True, notReverse = True): #
    myIndex = 1 if notReverse else 0
    myLoop = 0
    myRange = text if notReverse else reversed(text)
    for i in myRange:
        if i == splitChar: 
            myLoop+=1
            if myLoop==loop: 
                if reRight:
                    return text[myIndex:] if notReverse else text[-myIndex:] # 由index至右
                return text[:myIndex] if notReverse else text[:-myIndex] # 由左至index
        myIndex+=1

In [192]:
def sanmin(soup, infoData):
    # --- Price ---  (簡單暴力法, 待修改)
    str1 = soup.find('script', {'type':'application/ld+json'}).contents[0].strip()
    data = json.loads(str1)
    data['offers']['price']
    # --- image URL ---
    # 書封面 cdnec.sanmin.com.tw/product_images/
    # other cdnec.sanmin.com.tw/tryread/big/
    tmp = soup.find('div', class_='gallery-top').findAll('div', class_='swiper-slide')   # 大圖示欄
    imgURL = [tmp[0].img['src'][37:]]  # first img 
    imgURL.extend([i.img['src'][40:] for i in tmp[1:]])  # other img
    # --- Information ---
    tag = {'書名':'title','ISBN':'isbn','頁數':'page','出版社':'pub','作者':'author', '出版日期':'date'}
    findInfo = soup.find('meta', {'name':'description'})['content']
    for i in findInfo.split('，'):  # book info處理
        tmp =  i.split('：')
        if len(tmp)>2: tmp = [tmp[0], textClean(splitChar='：', text=i, loop=1)] # 標題, 內容(防止內容出現':'有問題)
        key = tag[tmp[0]] if (tmp[0] in tag) else None # 確標籤有在需求(tag)裡
        if key: infoData[key] = tmp[1]  # 資料內容
    # --- Intr ---
    # 標籤 ---
    tag = {
        "商品簡介": 'intr',
        "作者簡介": 'author',
        "目次": 'contents',
    }
    tabTitle_find = soup.findAll('div', class_='tab2')
    intrData = [{"title": None, "content": None} for i in range(len(tabTitle_find))]
    for index, i in enumerate(tabTitle_find):
        tmp = i.text.strip()  # ex: 商品簡介
        intrData[index]['title'] = tag[tmp] if (tmp in tag) else None  # ex: intr, else-> None 
    # 內容 ---    
    contents_find = soup.findAll('div', class_='productContent')
    for indexI, i in enumerate(contents_find):
        word = ''
        for j in i:
            tmp = j.string
            # if text is None -> loop data get string then join '\n' 
            word += f'\n{tmp}\n' if tmp!=None else "\n".join([k.string for k in j if k.string!=None])+'\n'
        intrData[indexI]['content'] = word
    return {
        'imgURL': imgURL,
        'info': infoData,
        'intr': intrData,
    }


In [26]:
soup = getHtml(method='get',url='https://www.sanmin.com.tw/Product/index/005135055')

requests -> 200


In [27]:

tmp = soup.find('div', class_='gallery-top')
if tmp:
    tmp = tmp.findAll('div', class_='swiper-slide')   # 大圖示欄
    # //cdnec.sanmin.com.tw/product_images/986/986502596.jpg
    imgURL = [tmp[0].img['src'].split('product_images/')[-1]]  # first img 
    # //cdnec.sanmin.com.tw/tryread/small/986/s_986502596_b1.jpg
    # //cdnec.sanmin.com.tw/tryread/big/986/986502596_b1.jpg
    imgURL.extend([i.img['src'].split('big/')[-1] for i in tmp[1:]])  # other img
else:
    imgURL = [soup.find('img', class_='mainImg')['src'].split('product_images/')[-1]]

In [28]:
imgURL

['986/986434014.jpg']

In [ ]:
# old indexSearch
imgURL = []
for i in soup.findAll('div', class_="swiper-slide"):
    tmp = i.img['src'].split('//cdnec.sanmin.com.tw/')[1]


labels = ['商品簡介', '作者簡介', '目次'] # code number
labels_ti = ['itIntr', 'auIntr', 'cons'] # labels dict title
# item Introduction、author Introduction、table contents
myLabels = []
boolIndex = []
for index, i in enumerate(soup.findAll('div', class_='IntroTab')):
    tmp = i.p.text
    if tmp in labels:
        myLabels.append(labels.index(tmp))
        boolIndex.append(index)


productContents = soup.findAll('div', class_='productContent')
word = {}
count = 0
for index, product in enumerate(productContents):  # contents(商品簡介、作者簡介...)
    if index in boolIndex:
        index = labels_ti[myLabels[count]]
        word[index] = []  # add (label:[])
        find_p = product.findAll('p') #有p 或 沒有([data, data ...] or [])
        len_p  = len(find_p)  # p有幾行
        mustLoop = True if len_p else False # 若沒p則不需要迴圈抓取
        for i in find_p if len_p else product:
            if i.find('br'): # 如果內容有br
                tmp = []
                if mustLoop:
                    for j in i:  # p 內容每行檢查
                        if str(j) != '<br/>': tmp.append(j.strip())    # 若文字不為br則放入list
                else:
                    if str(i) != '<br/>': tmp.append(i.strip())    # 若文字不為br則放入list
                word[index].append(tmp if len(tmp)>1 else tmp[0].strip())  # 如果清單才一個 則直接取出
            else:
                tmp = i.text.strip()
                if len(tmp): word[index].append(tmp)
        count +=1

In [30]:
import timeit
print(timeit.timeit('[]', number=10**7))
print(timeit.timeit('list()', number=10**7))

0.4227829000010388
1.2715858000010485


In [37]:
a=f'C:/Users/asias/Library/Application/%20Support/Google/Chrome/Default/History'

In [41]:
a.replace(f'/',f'\\')

'C:\\Users\\asias\\Library\\Application\\%20Support\\Google\\Chrome\\Default\\History'